In [54]:
import os
import json
import openai
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed

# Cargar las variables de entorno desde el archivo .env
load_dotenv(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\.env')

# Obtener la clave de API desde las variables de entorno
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API para OpenAI
openai.api_key = api_key
client = openai.Client()


def process_json_chunk_with_openai(json_chunks):
    """
    Procesa todos los chunks de JSON usando la API de OpenAI y devuelve el resultado.
    """
    # Unir todos los chunks para enviar una única solicitud con todo el contenido
    combined_chunks = "\n".join(json_chunks)
    
    # Enviar el mensaje final al modelo
    final_prompt = (
        "Has recibido varios fragmentos de datos JSON. Cada fragmento contiene datos que deben ser estructurados como tablas. "
        "A continuación se muestra la información completa. Procesa estos datos y devuelve todas las tablas relevantes."
        f"\n\n{combined_chunks}\n\n"
        "Estructura toda la información en formato de tablas claras y fáciles de leer."
    )
    
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=final_prompt,
        max_tokens=1000,
        temperature=0.5,


    )
    return response.choices[0].text.strip()

def split_json(json_data):
    """
    Divide los datos JSON en partes manejables para la API de OpenAI.
    """
    chunks = []
    max_chunk_size = 1500  # Tamaño máximo de cada fragmento
    current_chunk = ""
    json_data_str = json.dumps(json_data)
    
    for char in json_data_str:
        current_chunk += char
        if len(current_chunk) >= max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = ""
    
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

In [55]:

# Leer el archivo JSON completo
with open('C:/Users/jon.ruizcarrillo/Desktop/Projects/TFM/inno-dashboard-assistant-2024/Data_Extraction/Web_Scraping/Option_1.3/captured_responses1.json'
, 'r',encoding='latin1') as file:
   full_json_data = file.read()


In [56]:

# Dividir el JSON en partes
json_parts = split_json(full_json_data)


In [57]:
import json

def print_json_parts(json_parts, max_items_per_part=5):
    """
    Imprime las partes del JSON divididas para su revisión.
    
    :param json_parts: Lista de partes del JSON.
    :param max_items_per_part: Número máximo de elementos a mostrar por parte.
    """
    for i, part in enumerate(json_parts):
        print(f"\nParte {i + 1}:")
        
        # Si es una lista, muestra algunos elementos
        if isinstance(part, list):
            for j, item in enumerate(part[:max_items_per_part]):
                print(f"  Elemento {j + 1}: {json.dumps(item, indent=2)}")
            if len(part) > max_items_per_part:
                print("  ... (más elementos)")
        
        # Si es un diccionario, muestra algunas claves y valores
        elif isinstance(part, dict):
            for j, (key, value) in enumerate(list(part.items())[:max_items_per_part]):
                print(f"  Clave {j + 1}: {key}, Valor: {json.dumps(value, indent=2)}")
            if len(part) > max_items_per_part:
                print("  ... (más claves)")
        
        else:
            print(f"  Contenido: {json.dumps(part, indent=2)}")

# Uso de la función para imprimir las partes del JSON
print_json_parts(json_parts, max_items_per_part=5)



Parte 1:
  Contenido: "\"URL: https://wabi-west-us-c-primary-api.analysis.windows.net/public/reports/7040c84c-7a97-4457-bb73-9ae0b2073f86/modelsAndExploration?preferReadOnlySession=true\\nResponse Headers: {'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '24991', 'Content-Type': 'application/json; charset=utf-8', 'Content-Encoding': 'gzip', 'Access-Control-Expose-Headers': 'requestId,retry-after', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '31f7d07e-319e-a577-72e7-9787afbabafa', 'Timing-Allow-Origin': 'https://app.powerbi.com', 'Date': 'Wed, 11 Sep 2024 13:16:52 GMT'}\\nResponse Body: {\\\"models\\\":[{\\\"id\\\":356412,\\\"vsName\\\":\\\"sobe_wowvirtualserver\\\",\\\"dbName\\\":\\\"296a7eab-9be2-4e94-b0c8-e05cc0506381\\\",\\\"currentRefreshStartingTime\\\":\\\"\\\",\\\"refreshSchedule\\\":{\\\"importA

In [60]:
# Procesar todos los chunks en paralelo
results = []
with ThreadPoolExecutor(max_workers=5) as executor:  # Ajusta el número de trabajadores según tus necesidades
    futures = [executor.submit(process_json_chunk_with_openai, chunk) for chunk in json_parts]
    for future in as_completed(futures):
        results.append(future.result())

# Combinar todas las respuestas en una respuesta completa
combined_result = "\n".join(results)

# Mostrar o guardar los resultados
print(combined_result)


Tabla 1: Color
| Expresión | Literal | Valor |
|------------|---------|-------|
| Background | Solid   | #92c5de |
| Border     | Solid   | #67001f |
| Title      | Show    | False  |
| VisualHeader | Show  | False  |

Tabla 2: Visual Objects
| Background | Properties | Color | Show |
|------------|------------|-------|------|
| Solid      | Solid      | #92c5de | False |

Tabla 3: Filters
No hay datos disponibles.

Tabla 4: Query
| SemanticQueryDataShapeCommand | Query | Version | From | Type |
|-------------------------------|-------|---------|------|------|
| MainJobSheet                  | Main Job Sheet.Job Name | 2     | m    | 0    |

Tabla 5: Select
| Column | Expression | Property | Name |
|--------|------------|----------|------|
| Job Name | SourceRef.Source | Job Name |

Tabla 6: OrderBy
| Direction | Expression | Column | Property |
|-----------|------------|--------|----------|
| 1         | SourceRef.Source | Job Name |  |
Tabla 1: Primary Data Reduction
Data Volume: 3
P

In [61]:
with open("resultado.txt", "w", encoding="utf-8") as f:
    f.write(combined_result)